In [2]:
from transcribe import AudioRecorder

# Example usage in a Jupyter notebook:
audio_recorder = AudioRecorder(duration=5)  # Record for 5 seconds
transcript = audio_recorder.run()
print(transcript)

/Users/alexwang/Humming-Bird/drone/lib/python3.11/site-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_locati

Listening... Press Ctrl+C to stop.
Stopping...
Transcription:
 Hello hello I want to join the flight five years in here and hover for ten seconds in the land.
 Hello hello I want to join the flight five years in here and hover for ten seconds in the land.


In [38]:
pip install --upgrade openai

Note: you may need to restart the kernel to use updated packages.


In [3]:

from plan_agent import DroneCommandPlanner
planner = DroneCommandPlanner()
command = transcript
plan = planner.get_plan(command)
print("Drone Command Plan:\n")
print(plan)


Drone Command Plan:

1. Connect to the drone using the MavSDK library.
2. Take off and reach a stable altitude for flight.
3. Hold position in the air for five years.
4. Begin descending to land.
5. Hover in place for ten seconds.
6. Land the drone safely.


In [ ]:
from code_agent import DroneCommandExecutor
coder = MavSDKCodeGenerator

